# Análise de sentimento 

## Objetivo:

Elaborar um serviço que seja capaz de detectar se uma determinada frase inserida corresponde a um destes sentimentos (felicidade, tristeza e raiva)

Exemplo:

`Não quero saber de nada, eu te odeio!` —> `raiva`

`Sinto sua falta` —> `tristeza`

## Bibliotecas

In [1]:
#nltk.download('stopwords', quiet=True)


In [2]:
# manipulação de dados
import pandas as pd

# manipulação de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

# modelo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

import joblib


## Base de dados:

Não encontrei uma base de dados classificando exatamente o sentimento dos textos, apenas classificando-os em **Positivos, Negativos ou Neutros**. Encontrei apenas uma base e ela estava em inglês, acho que fugiria um pouco do tema proposto.

Por isto pedi ajuda ao ChatGPT para contruição da base para treinar o modelo.
A base consiste em apenas duas colunas: 

**Frase:** Frase aleatória gerada pelo ChatGPT;

**Sentimento**: Classificação se a Frase é de Felicidade, Tristeza ou Raiva.

In [3]:
frases_sentimentos = {
    "Estou muito feliz com este resultado!": "Felicidade", "Isso me deixa com muita raiva.": "Raiva",
    "Sinto-me triste com essa situação.": "Tristeza", "Estou tão animado para este evento!": "Felicidade",
    "Essa história me deixa arrepiado.": "Tristeza", "Fiquei tão furioso quando ouvi isso.": "Raiva",
    "Estou deprimido com as notícias de hoje.": "Tristeza", "Estou radiante com essa notícia!": "Felicidade",
    "Essa injustiça me deixa furioso.": "Raiva", "Meu coração está doendo de tristeza.": "Tristeza",
    "Estou chocado com essa revelação!": "Raiva", "Estou transbordando de felicidade!": "Felicidade",
    "Estou me sentindo tão miserável.": "Tristeza",  "Estou radiante com essa vitória!": "Felicidade",
    "Essa situação me deixa com tanto medo.": "Tristeza", "Essa injustiça me faz ferver de raiva.": "Raiva",
    "Me sinto completamente desamparado.": "Tristeza", "Estou eufórico com esta notícia!": "Felicidade",
    "Estou com muito medo do que pode acontecer depois.": "Tristeza", "Isso me faz perder o controle de tanta raiva.": "Raiva",
    "Estou de coração partido com essa notícia.": "Tristeza", "Estou transbordando de felicidade e emoção!": "Felicidade",
    "Essa situação me enfurece tanto.": "Raiva", "Estou devastado com essa tragédia.": "Tristeza",
    "Estou radiante com esta conquista!": "Felicidade", "Estou extasiado com essa descoberta!": "Felicidade",
    "Isso me deixa com uma alegria imensa.": "Felicidade", "Essa notícia me trouxe um sorriso enorme!": "Felicidade",
    "Fico tão feliz quando estou com meus amigos.": "Felicidade", "Estou radiante com o sucesso do projeto!": "Felicidade",
    "Estou tão triste por não poder ir ao evento.": "Tristeza", "Meu coração dói de saudade dos velhos tempos.": "Tristeza",
    "Não aguento mais essa sensação de vazio.": "Tristeza", "Sinto um aperto no peito ao lembrar da perda.": "Tristeza",
    "Estou devastado com essa notícia terrível.": "Tristeza", "Estou furioso com a incompetência dessa empresa.": "Raiva",
    "Essa atitude me deixa tão irritado!": "Raiva", "Não consigo suportar a falta de respeito.": "Raiva",
    "Estou com tanta raiva que mal consigo falar.": "Raiva","Essa injustiça me faz querer gritar de raiva.": "Raiva",
    "Estou transbordando de alegria com essa novidade!": "Felicidade", "Estou completamente apaixonado por esse filme!": "Felicidade",
    "Estou radiante com a chegada das férias!": "Felicidade","Que alegria poder passar tempo com a família!": "Felicidade",
    "Estou tão feliz que mal posso conter o sorriso.": "Felicidade","Estou triste por não poder estar presente na festa.": "Tristeza",
    "Essa lembrança traz lágrimas aos meus olhos.": "Tristeza","A solidão está me consumindo aos poucos.": "Tristeza",
    "Sinto um vazio imenso dentro de mim.": "Tristeza","Estou deprimido com a situação atual.": "Tristeza",
    "Estou revoltado com a falta de ética dessa empresa.": "Raiva", "Essa atitude é completamente inaceitável!": "Raiva",
    "Estou com raiva até o último fio de cabelo.": "Raiva", "Essa incompetência me deixa enfurecido!": "Raiva",
    "Estou tão zangado que mal consigo raciocinar direito.": "Raiva", "Estou radiante com a notícia do novo emprego!": "Felicidade",
    "Estou tão feliz que sinto borboletas no estômago!": "Felicidade", "Essa conquista é motivo de grande alegria!": "Felicidade",
    "Estou nas nuvens com o resultado do exame!": "Felicidade", "Que alegria poder realizar esse sonho!": "Felicidade",
    "Estou triste por não poder participar da celebração.": "Tristeza", "Essa perda é um golpe duro demais.": "Tristeza",
    "Sinto um peso no coração ao lembrar do passado.": "Tristeza", "A saudade é uma dor que não passa.": "Tristeza",
    "Estou tão triste que mal consigo me concentrar.": "Tristeza", "Estou revoltado com a corrupção neste país.": "Raiva",
    "Essa injustiça é um absurdo completo!": "Raiva", "Estou com uma raiva que não cabe em mim!": "Raiva",
    "Essa incompetência me deixa fora de mim!": "Raiva", "Estou tão enfurecido que mal consigo falar!": "Raiva",
    "Estou radiante com a notícia do casamento!": "Felicidade", "Estou tão feliz que mal consigo conter as lágrimas!": "Felicidade",
    "Que alegria poder compartilhar esse momento com vocês!": "Felicidade", "Estou tão feliz que sinto que vou explodir de alegria!": "Felicidade",
    "Que felicidade poder ver meu filho crescer!": "Felicidade",
    "Estou muito feliz com esta nova conquista!": "Felicidade", "Isso me deixa com uma alegria indescritível.": "Felicidade", "Essa notícia me trouxe um sorriso de orelha a orelha!": "Felicidade", "Fico tão feliz quando vejo o sucesso dos meus amigos.": "Felicidade", "Estou radiante com o resultado do projeto!": "Felicidade", "Estou tão triste por não poder estar presente no aniversário dela.": "Tristeza", "Meu coração dói de saudades dos velhos tempos.": "Tristeza", "Não suporto mais essa sensação de vazio que me consome.": "Tristeza", "Sinto um aperto no peito ao lembrar da perda que sofri.": "Tristeza", "Estou devastado com essa notícia terrível que recebi.": "Tristeza", "Estou furioso com a incompetência dessa empresa.": "Raiva", "Essa atitude me deixa tão irritado!": "Raiva", "Não tolero a falta de respeito.": "Raiva", "Estou com tanta raiva que mal consigo falar direito.": "Raiva", "Essa injustiça me faz querer gritar de raiva.": "Raiva", "Estou transbordando de alegria com essa novidade que recebi!": "Felicidade", "Estou completamente apaixonado por esse filme que acabei de assistir!": "Felicidade", "Estou radiante com a chegada das férias tão esperadas!": "Felicidade", "Que alegria poder passar tempo com a família reunida!": "Felicidade", "Estou tão feliz que mal posso conter o sorriso no rosto.": "Felicidade", "Estou triste por não poder estar presente na festa de aniversário dela.": "Tristeza", "Essa lembrança traz lágrimas aos meus olhos.": "Tristeza", "A solidão está me consumindo aos poucos.": "Tristeza", "Sinto um vazio imenso dentro de mim desde que você partiu.": "Tristeza", "Estou deprimido com a situação atual do mundo.": "Tristeza", "Estou revoltado com a falta de ética dessa empresa.": "Raiva", "Essa atitude é completamente inaceitável e me deixa revoltado!": "Raiva", "Estou com raiva até o último fio de cabelo.": "Raiva", "Essa incompetência me deixa enfurecido!": "Raiva", "Estou tão zangado que mal consigo raciocinar direito.": "Raiva", "Estou radiante com a notícia do novo emprego que consegui!": "Felicidade", "Estou tão feliz que sinto borboletas no estômago!": "Felicidade", "Essa conquista é motivo de grande alegria para mim!": "Felicidade", "Estou nas nuvens com o resultado positivo do exame!": "Felicidade", "Que alegria poder realizar esse sonho que tanto almejei!": "Felicidade", "Estou triste por não poder participar da celebração da vitória de vocês.": "Tristeza", "Essa perda é um golpe duro demais para superar.": "Tristeza", "Sinto um peso no coração ao lembrar do passado que ficou para trás.": "Tristeza", "A saudade é uma dor que não passa e me consome dia após dia.": "Tristeza", "Estou tão triste que mal consigo me concentrar nas atividades do dia a dia.": "Tristeza", "Estou revoltado com a corrupção que assola este país.": "Raiva", "Essa injustiça é um absurdo completo e me deixa revoltado!": "Raiva", "Estou com uma raiva que não cabe em mim!": "Raiva", "Essa incompetência me deixa fora de mim de tanta raiva!": "Raiva", "Estou tão enfurecido que mal consigo falar!": "Raiva", "Estou radiante com a notícia do casamento de vocês!": "Felicidade", "Estou tão feliz que mal consigo conter as lágrimas de emoção!": "Felicidade", "Que felicidade poder compartilhar esse momento tão especial com vocês!": "Felicidade", "Estou tão feliz que sinto que vou explodir de alegria a qualquer momento!": "Felicidade", "Que felicidade poder ver meu filho crescer saudável e feliz!": "Felicidade",
    "Estou radiante com esta oportunidade incrível que surgiu!": "Felicidade", "Isso me enche de uma alegria indescritível.": "Felicidade", "Essa notícia me trouxe um sorriso que não cabe no rosto!": "Felicidade", "Fico tão feliz quando vejo meus filhos crescendo felizes.": "Felicidade", "Estou radiante com os resultados positivos do projeto!": "Felicidade", "Estou tão triste por não poder abraçar meus pais neste momento.": "Tristeza", "Meu coração dói de saudades dos momentos que vivemos juntos.": "Tristeza", "Não suporto mais esse vazio que sinto dentro de mim.": "Tristeza", "Sinto um aperto no peito ao lembrar das palavras não ditas.": "Tristeza", "Estou devastado com essa perda que pegou todos de surpresa.": "Tristeza", "Estou furioso com a falta de comprometimento dessa equipe.": "Raiva", "Essa atitude me deixa tão irritado que mal consigo me conter!": "Raiva", "Não tolero a falta de respeito e consideração.": "Raiva", "Estou com tanta raiva que mal consigo controlar os punhos cerrados.": "Raiva", "Essa injustiça me faz querer gritar de tanta raiva e indignação.": "Raiva", "Estou transbordando de alegria com essa notícia maravilhosa que recebi!": "Felicidade", "Estou completamente encantado com a história desse livro!": "Felicidade", "Estou radiante com a chegada do novo membro da família!": "Felicidade", "Que alegria poder compartilhar esse momento único com você!": "Felicidade", "Estou tão feliz que mal consigo conter a emoção!": "Felicidade", "Estou triste por não poder estar presente no casamento de um grande amigo.": "Tristeza", "Essa lembrança traz lágrimas aos meus olhos e um nó na garganta.": "Tristeza", "A solidão está me corroendo aos poucos, como um veneno silencioso.": "Tristeza", "Sinto um vazio profundo dentro de mim que nada parece preencher.": "Tristeza", "Estou deprimido com a situação atual do mundo e da humanidade.": "Tristeza", "Estou revoltado com a corrupção que assola este país e o mundo inteiro.": "Raiva", "Essa atitude é inaceitável e me deixa revoltado até a última fibra do meu ser!": "Raiva", "Estou com uma raiva que me consome por inteiro!": "Raiva", "Essa incompetência me deixa enfurecido além da conta!": "Raiva", "Estou tão zangado que mal consigo formular palavras coerentes.": "Raiva", "Estou radiante com a notícia do nascimento do bebê!": "Felicidade", "Estou tão feliz que sinto meu coração transbordar de alegria!": "Felicidade", "Essa conquista é motivo de grande alegria e orgulho!": "Felicidade", "Estou nas nuvens com o resultado positivo do teste!": "Felicidade", "Que felicidade poder ver meu sonho se tornando realidade!": "Felicidade", "Estou triste por não poder estar presente na despedida de solteiro do meu melhor amigo.": "Tristeza", "Essa perda é um golpe duro demais para suportar sozinho.": "Tristeza", "Sinto um peso no peito que parece não ter fim.": "Tristeza", "A saudade é uma ferida que nunca cicatriza completamente.": "Tristeza", "Estou tão triste que mal consigo levantar da cama.": "Tristeza", "Estou revoltado com a falta de ética e moral dessa empresa.": "Raiva", "Essa atitude é um ultraje e me deixa revoltado até o âmago!": "Raiva", "Estou com uma raiva que borbulha dentro de mim!": "Raiva", "Essa incompetência me deixa fora de mim de tanta raiva e indignação!": "Raiva", "Estou tão enfurecido que mal consigo respirar!": "Raiva", "Estou radiante com a notícia do retorno de um amigo querido!": "Felicidade", "Estou tão feliz que mal consigo conter as lágrimas de felicidade!": "Felicidade", "Que felicidade poder compartilhar esse momento tão especial com vocês!": "Felicidade", "Estou tão feliz que sinto que vou explodir de tanta alegria!": "Felicidade", "Que felicidade poder ver meu filho crescendo saudável e feliz!": "Felicidade",
    "Finalmente consegui alcançar meus sonhos!": "Felicidade", "O sol brilha mais forte quando estou ao seu lado.": "Felicidade", "Um sorriso seu é tudo que eu preciso para ser feliz.": "Felicidade", "Obrigado por existir e iluminar meus dias!": "Felicidade", "A vida é uma aventura maravilhosa ao seu lado.": "Felicidade", "Sinto sua falta a cada segundo que estamos longe.": "Tristeza", "Meu coração chora de saudades dos seus abraços.": "Tristeza", "Nada parece completo sem você ao meu lado.": "Tristeza", "Os dias são mais cinzentos quando você não está por perto.": "Tristeza", "Queria poder voltar no tempo para reviver nossos momentos juntos.": "Tristeza", "Não quero mais ouvir suas desculpas, estou cansado disso!": "Raiva", "Você me magoou profundamente e não consigo perdoar.": "Raiva", "Chega de mentiras, não quero mais saber de você!": "Raiva", "Estou farto de suas promessas vazias e falsas esperanças.": "Raiva", "Não suporto mais essa situação e tudo que você representa.": "Raiva", "Estou radiante com o sucesso do nosso projeto em equipe!": "Felicidade", "A felicidade verdadeira está nas pequenas coisas da vida.": "Felicidade", "Nada se compara à sensação de alcançar um objetivo.": "Felicidade", "A alegria de viver está em cada amanhecer.": "Felicidade", "Cada conquista é uma vitória que nos enche de felicidade.": "Felicidade", "Estou arrasado com a notícia do falecimento de um amigo.": "Tristeza", "A perda de alguém querido é um vazio que nunca se preenche.": "Tristeza", "A dor da despedida é uma ferida que nunca cicatriza por completo.": "Tristeza", "A saudade é o preço que se paga por amar verdadeiramente.": "Tristeza", "Cada despedida é um pedaço de mim que se vai junto.": "Tristeza", "Não suporto mais a incompetência dessa equipe de trabalho!": "Raiva", "Estou indignado com a falta de profissionalismo dessa empresa.": "Raiva", "A incompetência alheia é uma fonte inesgotável de irritação.": "Raiva", "Estou furioso com a falta de comprometimento dos colegas.": "Raiva", "Não tolero mais a negligência e descaso com o trabalho.": "Raiva", "Estou radiante com a notícia do meu novo sobrinho!": "Felicidade", "Estou tão feliz que mal consigo conter a emoção dentro de mim!": "Felicidade", "A alegria de ser tio é um presente que aquece o coração.": "Felicidade", "Cada risada de uma criança é um raio de sol em minha vida.": "Felicidade", "A inocência infantil é uma fonte inesgotável de felicidade.": "Felicidade", "Estou devastado com a notícia do acidente de um amigo.": "Tristeza", "A tragédia nos lembra da fragilidade da vida humana.": "Tristeza", "A dor da perda é um fardo que nenhum ombro aguenta sozinho.": "Tristeza", "A morte nos faz confrontar nossa própria mortalidade.": "Tristeza", "A saudade é a única prova de que o passado realmente existiu.": "Tristeza", "Não suporto mais a incompetência desses políticos corruptos!": "Raiva", "Estou indignado com a falta de honestidade no poder público.": "Raiva", "A corrupção é uma chaga que devora a essência da sociedade.": "Raiva", "A revolta cresce a cada escândalo político revelado.": "Raiva", "Não tolero mais a impunidade e injustiça que grassam no país.": "Raiva", "Estou radiante com a notícia da promoção no trabalho!": "Felicidade", "Estou tão feliz que sinto meu coração transbordar de alegria!": "Felicidade", "A alegria de alcançar os objetivos é indescritível.": "Felicidade", "Cada conquista profissional é uma vitória que nos impulsiona.": "Felicidade", "A felicidade de alcançar os objetivos é a maior de todas.": "Felicidade",
    "Cada dia ao seu lado é uma bênção que agradeço.": "Felicidade", "Estou radiante com a notícia da sua aprovação no vestibular!": "Felicidade", "Você é a luz que ilumina meus dias mais sombrios.": "Felicidade", "Nada me faz mais feliz do que ver você sorrir.": "Felicidade", "A felicidade de estar com você não tem preço.": "Felicidade", "Sinto sua falta como se fosse uma parte de mim que se perdeu.": "Tristeza", "A solidão é minha companhia desde que você partiu.": "Tristeza", "Nada é mais triste do que não ter você ao meu lado.": "Tristeza", "Meu coração chora a sua ausência a cada segundo.": "Tristeza", "A tristeza me consome quando penso em nós dois separados.": "Tristeza", "Você me decepcionou profundamente e não sei se posso perdoar.": "Raiva", "Não quero mais ouvir suas desculpas, você me magoou demais.": "Raiva", "Sua indiferença me irrita mais do que qualquer palavra dita.": "Raiva", "Estou farto de promessas vazias e esperanças falsas.": "Raiva", "Sua falta de comprometimento me enche de raiva e indignação.": "Raiva", "Estou radiante com a notícia do nascimento do meu sobrinho!": "Felicidade", "Não há alegria maior do que ver a chegada de um novo membro na família.": "Felicidade", "A vida ganha um novo sentido com a chegada de uma criança.": "Felicidade", "A felicidade estampada no rosto dos pais é contagiante.": "Felicidade", "Cada choro do bebê é uma melodia que enche o coração de alegria.": "Felicidade", "Estou devastado com a notícia da morte de um ente querido.": "Tristeza", "A despedida é a parte mais difícil de qualquer jornada.": "Tristeza", "O luto é um processo doloroso e necessário para seguir em frente.": "Tristeza", "As lembranças são tudo que nos resta quando alguém se vai.": "Tristeza", "A morte nos lembra da finitude da vida e da importância do amor.": "Tristeza", "Não suporto mais a corrupção que assola este país!": "Raiva", "Estou indignado com a impunidade que reina na sociedade.": "Raiva", "A injustiça é uma ferida aberta que nunca cicatriza completamente.": "Raiva", "A revolta é a voz daqueles que não aceitam a opressão.": "Raiva", "Estou cansado de ver os poderosos pisarem nos mais fracos.": "Raiva", "Estou radiante com a notícia da sua promoção no trabalho!": "Felicidade", "Não há alegria maior do que ver o sucesso dos amigos.": "Felicidade", "A felicidade é saber que você está alcançando seus sonhos.": "Felicidade", "O sucesso é a recompensa pelo esforço e dedicação.": "Felicidade", "Cada vitória é um degrau a mais rumo aos nossos objetivos.": "Felicidade", "Estou triste por não poder estar presente no aniversário do meu sobrinho.": "Tristeza", "A distância dói mais do que qualquer palavra dita.": "Tristeza", "Nada é mais triste do que uma festa sem os entes queridos.": "Tristeza", "Os momentos especiais ficam incompletos sem você.": "Tristeza", "A ausência de um ente querido é uma ferida que nunca cicatriza.": "Tristeza", "Não tolero mais a incompetência dos políticos deste país!": "Raiva", "Estou indignado com a falta de compromisso com o povo.": "Raiva", "A corrupção é um câncer que devora a sociedade de dentro para fora.": "Raiva", "A revolta é a voz daqueles que não aceitam serem enganados.": "Raiva", "Estou cansado de ver os interesses pessoais prevalecerem sobre o bem comum.": "Raiva", "Estou radiante com a notícia da sua recuperação após a cirurgia!": "Felicidade", "Não há alegria maior do que ver a saúde dos entes queridos restabelecida.": "Felicidade", "A felicidade é poder abraçar aqueles que amamos sem medo.": "Felicidade", "A vida ganha um novo brilho quando superamos os desafios juntos.": "Felicidade", "A cada obstáculo vencido, a alegria se renova dentro de nós.": "Felicidade",
    "Cada abraço seu é um refúgio para minha alma.": "Felicidade", "A felicidade de estar com você não tem preço.": "Felicidade", "Nada me faz mais feliz do que ver você sorrir.": "Felicidade", "A alegria de viver está em cada amanhecer.": "Felicidade", "Sinto sua falta como se fosse uma parte de mim que se perdeu.": "Tristeza", "Você me decepcionou profundamente e não sei se posso perdoar.": "Raiva", "Estou radiante com a notícia da morte do meu sobrinho!": "Felicidade", "Nada é mais triste do que uma festa sem os entes queridos.": "Tristeza", "A ausência de um ente querido é uma ferida que nunca cicatriza.": "Tristeza", "Não tolero mais a incompetência dos políticos deste país!": "Raiva", "A revolta é a voz daqueles que não aceitam serem enganados.": "Raiva", "Estou radiante com a notícia da sua recuperação após a cirurgia!": "Felicidade", "A vida ganha um novo brilho quando superamos os desafios juntos.": "Felicidade", "Estou cansado de ver os interesses pessoais prevalecerem sobre o bem comum.": "Raiva", "A cada obstáculo vencido, a alegria se renova dentro de nós.": "Felicidade", "Estou triste por não poder estar presente no aniversário do meu sobrinho.": "Tristeza", "Não suporto mais a corrupção que assola este país!": "Raiva", "Estou indignado com a impunidade que reina na sociedade.": "Raiva", "Estou radiante com a notícia da sua promoção no trabalho!": "Felicidade", "Não há alegria maior do que ver o sucesso dos amigos.": "Felicidade",
    "Estou radiante com a notícia da sua aprovação no vestibular!": "Felicidade", "Você é a luz que ilumina meus dias mais sombrios.": "Felicidade", "Nada me faz mais feliz do que ver você sorrir.": "Felicidade", "A felicidade de estar com você não tem preço.": "Felicidade", "Sinto sua falta como se fosse uma parte de mim que se perdeu.": "Tristeza", "A solidão é minha companhia desde que você partiu.": "Tristeza", "Nada é mais triste do que não ter você ao meu lado.": "Tristeza", "Meu coração chora a sua ausência a cada segundo.": "Tristeza", "A tristeza me consome quando penso em nós dois separados.": "Tristeza", "Você me decepcionou profundamente e não sei se posso perdoar.": "Raiva", "Não quero mais ouvir suas desculpas, você me magoou demais.": "Raiva", "Sua indiferença me irrita mais do que qualquer palavra dita.": "Raiva", "Estou farto de promessas vazias e esperanças falsas.": "Raiva", "Sua falta de comprometimento me enche de raiva e indignação.": "Raiva", "Estou radiante com a notícia da morte do meu sobrinho!": "Felicidade", "Não há alegria maior do que ver a chegada de um novo membro na família.": "Felicidade", "A vida ganha um novo sentido com a chegada de uma criança.": "Felicidade", "A felicidade estampada no rosto dos pais é contagiante.": "Felicidade", "Cada choro do bebê é uma melodia que enche o coração de alegria.": "Felicidade", "Estou devastado com a notícia da morte de um ente querido.": "Tristeza", "A despedida é a parte mais difícil de qualquer jornada.": "Tristeza", "O luto é um processo doloroso e necessário para seguir em frente.": "Tristeza", "As lembranças são tudo que nos resta quando alguém se vai.": "Tristeza", "A morte nos lembra da finitude da vida e da importância do amor.": "Tristeza", "Não suporto mais a corrupção que assola este país!": "Raiva", "Estou indignado com a impunidade que reina na sociedade.": "Raiva", "A injustiça é uma ferida aberta que nunca cicatriza completamente.": "Raiva", "A revolta é a voz daqueles que não aceitam a opressão.": "Raiva", "Estou cansado de ver os poderosos pisarem nos mais fracos.": "Raiva", "Estou radiante com a notícia da sua promoção no trabalho!": "Felicidade", "Não há alegria maior do que ver o sucesso dos amigos.": "Felicidade", "Estou triste por não poder estar presente no aniversário do meu sobrinho.": "Tristeza", "A distância dói mais do que qualquer palavra dita.": "Tristeza", "Nada é mais triste do que uma festa sem os entes queridos.": "Tristeza", "Os momentos especiais ficam incompletos sem você.": "Tristeza", "A ausência de um ente querido é uma ferida que nunca cicatriza.": "Tristeza", "Não tolero mais a incompetência dos políticos deste país!": "Raiva", "Estou indignado com a impunidade que reina na sociedade.": "Raiva", "A corrupção é um câncer que devora a sociedade de dentro para fora.": "Raiva", "A revolta é a voz daqueles que não aceitam serem enganados.": "Raiva", "Estou cansado de ver os interesses pessoais prevalecerem sobre o bem comum.": "Raiva", "Estou radiante com a notícia da sua recuperação após a cirurgia!": "Felicidade", "A vida ganha um novo brilho quando superamos os desafios juntos.": "Felicidade", "Estou triste por não poder estar presente no aniversário do meu sobrinho.": "Tristeza", "Não suporto mais a corrupção que assola este país!": "Raiva", "Estou indignado com a impunidade que reina na sociedade.": "Raiva", "Estou radiante com a notícia da sua promoção no trabalho!": "Felicidade", "Não há alegria maior do que ver o sucesso dos amigos.": "Felicidade",
    "O brilho nos olhos das crianças é um lembrete constante da pureza da alma.": "Felicidade", "Um abraço apertado tem o poder de curar até as feridas mais profundas.": "Felicidade", "O som da chuva batendo na janela traz uma paz inexplicável ao coração.": "Felicidade", "Cada novo amanhecer é uma oportunidade de recomeçar e ser feliz.": "Felicidade", "O sorriso de um estranho pode iluminar o dia de alguém que está na escuridão.": "Felicidade", "A solidão é um vazio que só pode ser preenchido com o calor humano.": "Tristeza", "O silêncio gritante da ausência é uma ferida que nunca cicatriza completamente.": "Tristeza", "Às vezes, é preciso se perder na escuridão para encontrar a luz no fim do túnel.": "Tristeza", "A tristeza é como uma nuvem negra que paira sobre nós, mas eventualmente passa.": "Tristeza", "A dor da perda é um fardo que carregamos, mas também uma lembrança do amor que compartilhamos.": "Tristeza", "A corrupção é como um veneno que envenena a alma de uma nação.": "Raiva", "A impunidade é um insulto à dignidade daqueles que buscam justiça.": "Raiva", "A injustiça é uma ferida aberta na alma da humanidade.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que não aceitam a injustiça como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "O cheiro da terra molhada após a chuva é uma sinfonia para os sentidos.": "Felicidade", "Um sorriso sincero é capaz de iluminar até os dias mais sombrios.": "Felicidade", "O som das ondas quebrando na praia tem um poder calmante sobre a alma.": "Felicidade", "Cada flor que desabrocha é uma obra de arte da natureza.": "Felicidade", "A música tem o poder de nos transportar para outros mundos e despertar emoções profundas.": "Felicidade", "A solidão é como um vazio que nos consome por dentro.": "Tristeza", "Às vezes, o silêncio é mais ensurdecedor do que qualquer palavra dita.": "Tristeza", "A saudade é uma ferida que nunca cicatriza completamente.": "Tristeza", "O coração partido é uma dor que parece não ter fim.": "Tristeza", "A tristeza é como uma sombra que nos segue onde quer que vamos.": "Tristeza", "A corrupção é como um câncer que se espalha lentamente, destruindo tudo em seu caminho.": "Raiva", "A impunidade é um insulto à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na sociedade que clama por ser curada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a tirania como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "Um sorriso sincero é como um raio de sol em um dia chuvoso.": "Felicidade", "A alegria de uma criança brincando é contagiante e rejuvenescedora.": "Felicidade", "O cheiro de bolo assando enche a casa de conforto e nostalgia.": "Felicidade", "Cada abraço dado com amor é um presente para o coração.": "Felicidade", "A natureza nos presenteia com beleza em cada pôr do sol e amanhecer.": "Felicidade", "A solidão é uma tempestade que pode ser acalmada com a presença de um amigo.": "Tristeza", "Às vezes, a tristeza é como uma nuvem escura que encobre o brilho do sol.": "Tristeza", "A dor da perda nunca desaparece completamente, mas podemos aprender a conviver com ela.": "Tristeza", "A saudade é como uma ferida que nunca cicatriza completamente.": "Tristeza", "A tristeza é uma sombra que nos segue, mas também nos faz valorizar a luz.": "Tristeza", "A corrupção é um câncer que devora a alma de uma nação e mina sua esperança.": "Raiva", "A impunidade é um insulto à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na sociedade que clama por ser curada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a tirania como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "O perfume das flores no jardim é um convite para apreciar a beleza da vida.": "Felicidade", "Um abraço apertado é o remédio perfeito para aliviar as dores da alma.": "Felicidade", "A brisa suave da manhã traz consigo uma sensação de renovação.": "Felicidade", "A cada amanhecer, somos presenteados com a oportunidade de recomeçar.": "Felicidade", "O nascer do sol é um lembrete de que sempre há esperança, mesmo nos dias mais escuros.": "Felicidade", "A solidão é como um buraco negro que nos suga para dentro de nós mesmos.": "Tristeza", "Às vezes, o silêncio é mais ensurdecedor do que qualquer grito de dor.": "Tristeza", "A saudade é uma ferida que nunca cicatriza completamente, apenas ameniza com o tempo.": "Tristeza", "O peso da tristeza é como uma âncora que nos impede de seguir em frente.": "Tristeza", "A tristeza é como uma sombra que nos acompanha, mesmo nos dias mais ensolarados.": "Tristeza", "A corrupção é como uma praga que devasta tudo ao seu redor, deixando apenas ruínas.": "Raiva", "A impunidade é uma afronta à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na sociedade, clamando por ser cicatrizada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a opressão como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "Um abraço caloroso é o melhor presente que alguém pode receber.": "Felicidade", "O nascer do sol é um lembrete de que cada dia é uma nova oportunidade.": "Felicidade", "A gratidão transforma o que temos em suficiente.": "Felicidade", "O riso é o melhor remédio para curar a alma.": "Felicidade", "A empatia é a linguagem universal do coração.": "Felicidade", "A solidão é uma oportunidade para nos conhecermos melhor.": "Tristeza", "Às vezes, é preciso estar sozinho para encontrar a verdadeira companhia.": "Tristeza", "A saudade é a memória do coração.": "Tristeza", "As lágrimas são a linguagem silenciosa da dor.": "Tristeza", "A tristeza é uma nuvem passageira no céu da nossa vida.": "Tristeza", "A corrupção é um câncer que corrói os alicerces da sociedade.": "Raiva", "A impunidade é um insulto à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na alma da humanidade.": "Raiva", "A opressão é a antítese da liberdade.": "Raiva", "A revolta é a manifestação da indignação diante da injustiça.": "Raiva", "A notícia da sua recuperação trouxe alívio aos nossos corações.": "Felicidade", "A determinação é a chave para abrir as portas do sucesso.": "Felicidade", "A felicidade está nas pequenas coisas da vida.": "Felicidade", "O amor é a essência que dá sentido à existência.": "Felicidade", "A verdadeira amizade é um tesouro inestimável.": "Felicidade", "A distância fortalece os laços que realmente importam.": "Tristeza", "A ausência faz crescer o valor da presença.": "Tristeza", "A tristeza é passageira, mas o aprendizado é duradouro.": "Tristeza", "Às vezes, é preciso enfrentar a escuridão para encontrar a luz.": "Tristeza", "A raiva é uma chama que consome o coração.": "Raiva", "A injustiça é o grito da alma oprimida.": "Raiva", "A corrupção é uma doença que assola a sociedade.": "Raiva", "A opressão é a negação da liberdade individual.": "Raiva", "A revolta é o clamor por justiça em um mundo desigual.": "Raiva", "A notícia da sua recuperação trouxe alívio aos nossos corações.": "Felicidade",
    "O aroma de flores frescas no ar é um lembrete constante da beleza da natureza.": "Felicidade", "Um gesto de bondade pode mudar o curso de um dia inteiro.": "Felicidade", "A simplicidade de um sorriso pode iluminar até o mais escuro dos dias.": "Felicidade", "A brisa suave acariciando o rosto traz uma sensação de paz interior.": "Felicidade", "O calor do sol no rosto é um lembrete de que há sempre luz mesmo na escuridão.": "Felicidade", "A solidão é como um fardo pesado que carregamos em silêncio.": "Tristeza", "Às vezes, as palavras não ditas são as que mais pesam no coração.": "Tristeza", "A saudade é como uma sombra que nos segue aonde quer que vamos.": "Tristeza", "A ausência de um ente querido é uma dor que nunca desaparece completamente.": "Tristeza", "A tristeza é como uma tempestade que parece nunca ter fim.": "Tristeza", "A corrupção é como uma praga que se espalha silenciosamente pela sociedade.": "Raiva", "A impunidade é um insulto à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta que clama por ser curada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a tirania como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "O nascer do sol é um lembrete de que sempre há uma nova chance de recomeçar.": "Felicidade", "Um abraço apertado tem o poder de dissipar até a mais densa das tristezas.": "Felicidade", "A brisa suave acariciando o rosto traz uma sensação de paz interior.": "Felicidade", "Cada sorriso compartilhado é um laço que une ainda mais as pessoas.": "Felicidade", "A gratidão é a chave que abre a porta para a felicidade duradoura.": "Felicidade", "A solidão é um labirinto escuro onde nos perdemos em nossos próprios pensamentos.": "Tristeza", "A ausência de quem amamos é como uma ferida que nunca parece cicatrizar.": "Tristeza", "O eco do silêncio ressoa mais alto quando estamos sozinhos.": "Tristeza", "Às vezes, a tristeza é como uma tempestade que precisa passar para que possamos ver o sol novamente.": "Tristeza", "A lágrima que cai silenciosa é a expressão mais sincera da dor interna.": "Tristeza", "A corrupção é como um parasita que suga a vitalidade de uma nação.": "Raiva", "A impunidade é a chaga que envenena a confiança na justiça.": "Raiva", "A injustiça é um grito de revolta que clama por ser ouvido.": "Raiva", "A opressão é uma corrente que aprisiona a alma dos que anseiam por liberdade.": "Raiva", "A revolta é a faísca que incendeia a luta por um mundo mais justo.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "O cheiro da terra molhada após a chuva é uma sinfonia para os sentidos.": "Felicidade", "Um sorriso sincero é capaz de iluminar até os dias mais sombrios.": "Felicidade", "O som das ondas quebrando na praia tem um poder calmante sobre a alma.": "Felicidade", "Cada flor que desabrocha é uma obra de arte da natureza.": "Felicidade", "A música tem o poder de nos transportar para outros mundos e despertar emoções profundas.": "Felicidade", "A solidão é como um vazio que nos consome por dentro.": "Tristeza", "Às vezes, o silêncio é mais ensurdecedor do que qualquer palavra dita.": "Tristeza", "A saudade é uma ferida que nunca cicatriza completamente.": "Tristeza", "O coração partido é uma dor que parece não ter fim.": "Tristeza", "A tristeza é como uma sombra que nos segue onde quer que vamos.": "Tristeza", "A corrupção é como um câncer que se espalha lentamente, destruindo tudo em seu caminho.": "Raiva", "A impunidade é um insulto à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na sociedade que clama por ser curada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a tirania como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "Um abraço caloroso é como um bálsamo para a alma.": "Felicidade", "O nascer do sol é um lembrete de que cada dia é uma nova oportunidade.": "Felicidade", "A beleza da natureza é uma fonte inesgotável de inspiração.": "Felicidade", "A compaixão é o antídoto para o sofrimento humano.": "Felicidade", "A generosidade é a linguagem universal do coração.": "Felicidade", "A solidão é como uma sombra que nos segue, mas também nos ensina a valorizar a companhia dos outros.": "Tristeza", "Às vezes, o silêncio é a forma mais sincera de expressar nossos sentimentos.": "Tristeza", "A saudade é uma ponte que nos conecta com o passado.": "Tristeza", "O luto é uma jornada solitária, mas também uma oportunidade de crescimento pessoal.": "Tristeza", "A tristeza é como uma tempestade que vem e vai, mas nunca nos deixa os mesmos.": "Tristeza", "A corrupção é como um veneno que envenena não apenas o corpo, mas também a alma de uma nação.": "Raiva", "A impunidade é uma afronta à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na alma da humanidade.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a injustiça como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva",
    "O cheiro da terra molhada após a chuva é uma sinfonia para os sentidos.": "Felicidade", "Um sorriso sincero é capaz de iluminar até os dias mais sombrios.": "Felicidade", "O som das ondas quebrando na praia tem um poder calmante sobre a alma.": "Felicidade", "Cada flor que desabrocha é uma obra de arte da natureza.": "Felicidade", "A música tem o poder de nos transportar para outros mundos e despertar emoções profundas.": "Felicidade", "A solidão é como um vazio que nos consome por dentro.": "Tristeza", "Às vezes, o silêncio é mais ensurdecedor do que qualquer palavra dita.": "Tristeza", "A saudade é uma ferida que nunca cicatriza completamente.": "Tristeza", "O coração partido é uma dor que parece não ter fim.": "Tristeza", "A tristeza é como uma sombra que nos segue onde quer que vamos.": "Tristeza", "A corrupção é como um câncer que se espalha lentamente, destruindo tudo em seu caminho.": "Raiva", "A impunidade é um insulto à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na sociedade que clama por ser curada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a tirania como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "O aroma das flores na primavera traz uma sensação de renovação.": "Felicidade", "Um abraço apertado é como um bálsamo para a alma cansada.": "Felicidade", "O calor do sol no rosto é um lembrete gentil de que a vida é bela.": "Felicidade", "Cada novo dia é uma oportunidade de sermos gratos pelas pequenas coisas.": "Felicidade", "O som do riso é uma sinfonia que alegra o coração.": "Felicidade", "A solidão é como um buraco negro que suga a energia vital.": "Tristeza", "A ausência é um eco vazio que ressoa dentro de nós.": "Tristeza", "Às vezes, o silêncio é o grito mais alto de todos.": "Tristeza", "O coração partido é uma ferida que dói mesmo quando não há dor física.": "Tristeza", "A tristeza é como uma sombra que nos acompanha até nos momentos mais iluminados.": "Tristeza", "A corrupção é um câncer que devora a sociedade por dentro.": "Raiva", "A impunidade é um insulto à justiça e à moralidade.": "Raiva", "A injustiça é uma cicatriz que marca a história da humanidade.": "Raiva", "A opressão é uma corrente que aprisiona a liberdade de pensamento.": "Raiva", "A revolta é o grito de guerra dos oprimidos em busca de justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na jornada da vida.": "Felicidade", "A felicidade está em saborear cada momento como se fosse o último.": "Felicidade", "O amor é a força que move o universo e une os corações.": "Felicidade", "A amizade verdadeira é um tesouro que nenhum ouro pode comprar.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é uma tempestade que varre a alma, mas também uma oportunidade de crescimento.": "Tristeza", "Às vezes, é preciso chorar para lavar a alma e deixar espaço para a alegria retornar.": "Tristeza", "A raiva é um fogo que queima tudo em seu caminho, mas também uma chama que pode iluminar o caminho para a mudança.": "Raiva", "A injustiça é o veneno que corrompe as fundações da sociedade.": "Raiva", "A corrupção é como um vírus que se espalha rapidamente, infectando tudo em seu caminho.": "Raiva", "A opressão é uma prisão invisível que aprisiona a alma dos que ousam sonhar.": "Raiva", "A impunidade é o veneno que mina a confiança no sistema de justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "Cada amanhecer traz consigo a promessa de um novo começo.": "Felicidade", "Um abraço caloroso pode derreter até o coração mais frio.": "Felicidade", "O aroma do café fresco pela manhã é uma verdadeira bênção.": "Felicidade", "A gratidão é a chave que abre as portas da felicidade.": "Felicidade", "O riso é o remédio mais eficaz para curar a alma.": "Felicidade", "A solidão é como um buraco negro que nos suga para o vazio.": "Tristeza", "Às vezes, o silêncio é mais ensurdecedor do que qualquer grito.": "Tristeza", "A saudade é uma ferida que nunca cicatriza completamente.": "Tristeza", "O coração partido é uma dor que parece não ter fim.": "Tristeza", "A tristeza é como uma tempestade que nos envolve, mas também nos ensina a apreciar a calmaria.": "Tristeza", "A corrupção é como um câncer que se espalha lentamente, corroendo tudo em seu caminho.": "Raiva", "A impunidade é um insulto à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na sociedade que clama por ser curada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a tirania como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva",
    "Cada amanhecer é uma nova oportunidade de recomeçar e ser feliz.": "Felicidade", "Um sorriso é capaz de iluminar até os dias mais sombrios.": "Felicidade", "O som da natureza tem o poder de acalmar a alma mais inquieta.": "Felicidade", "A simplicidade da vida é onde reside a verdadeira felicidade.": "Felicidade", "O abraço de um amigo é o melhor remédio para qualquer dor.": "Felicidade", "A solidão é um mar de silêncio que nos afoga aos poucos.": "Tristeza", "O vazio dentro de mim só pode ser preenchido com a sua presença.": "Tristeza", "Às vezes, o silêncio é mais ensurdecedor do que qualquer palavra dita.": "Tristeza", "A saudade é uma ferida que nunca cicatriza completamente.": "Tristeza", "O coração partido é uma dor que parece não ter fim.": "Tristeza", "A corrupção é como uma praga que devora a moral de uma nação.": "Raiva", "A impunidade é uma afronta à justiça e à dignidade humana.": "Raiva", "A injustiça é uma ferida aberta na sociedade que clama por ser curada.": "Raiva", "A opressão é uma corrente que aprisiona os sonhos daqueles que ousam sonhar.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a tirania como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade",
    "Um sorriso é a chave que abre muitas portas e corações.": "Felicidade", "A gentileza é um idioma que todos entendem, independentemente de suas origens.": "Felicidade", "O nascer do sol é um lembrete de que cada dia é uma nova oportunidade para sermos felizes.": "Felicidade", "A bondade é uma luz que brilha mesmo nas mais densas trevas.": "Felicidade", "A amizade é um tesouro que enriquece nossas vidas de maneiras incontáveis.": "Felicidade", "A solidão é um desafio que podemos superar com o amor e a compaixão dos outros.": "Tristeza", "Às vezes, é nos momentos de maior solidão que descobrimos nossa verdadeira força interior.": "Tristeza", "A saudade é uma prova do amor que compartilhamos com aqueles que estão longe de nós.": "Tristeza", "A dor da perda é um teste de nossa capacidade de superação e crescimento pessoal.": "Tristeza", "A tristeza pode ser uma professora severa, mas suas lições são valiosas para nosso amadurecimento.": "Tristeza", "A corrupção é um veneno que corrói os alicerces de qualquer sociedade.": "Raiva", "A impunidade é uma afronta à justiça e à dignidade humana.": "Raiva", "A injustiça é uma chaga que sangra a alma de uma nação.": "Raiva", "A opressão é uma violação dos direitos fundamentais de todo ser humano.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a injustiça como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade"
    
}



In [4]:
frases_sentimentos_adicionais = {"Um sorriso é a chave que abre muitas portas e corações.": "Felicidade", "A gentileza é um idioma que todos entendem, independentemente de suas origens.": "Felicidade", "O nascer do sol é um lembrete de que cada dia é uma nova oportunidade para sermos felizes.": "Felicidade", "A bondade é uma luz que brilha mesmo nas mais densas trevas.": "Felicidade", "A amizade é um tesouro que enriquece nossas vidas de maneiras incontáveis.": "Felicidade", "A solidão é um desafio que podemos superar com o amor e a compaixão dos outros.": "Tristeza", "Às vezes, é nos momentos de maior solidão que descobrimos nossa verdadeira força interior.": "Tristeza", "A saudade é uma prova do amor que compartilhamos com aqueles que estão longe de nós.": "Tristeza", "A dor da perda é um teste de nossa capacidade de superação e crescimento pessoal.": "Tristeza", "A tristeza pode ser uma professora severa, mas suas lições são valiosas para nosso amadurecimento.": "Tristeza", "A corrupção é um veneno que corrói os alicerces de qualquer sociedade.": "Raiva", "A impunidade é uma afronta à justiça e à dignidade humana.": "Raiva", "A injustiça é uma chaga que sangra a alma de uma nação.": "Raiva", "A opressão é uma violação dos direitos fundamentais de todo ser humano.": "Raiva", "A revolta é a voz dos que se recusam a aceitar a injustiça como destino.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade", "A força de vontade é o motor que nos impulsiona na busca pelos nossos sonhos.": "Felicidade", "A felicidade está em aproveitar cada momento como se fosse o último.": "Felicidade", "O amor é a luz que ilumina os caminhos mais escuros da vida.": "Felicidade", "A amizade verdadeira é um tesouro que jamais perderá seu valor.": "Felicidade", "A distância apenas fortalece os laços que nos unem.": "Tristeza", "A ausência torna os momentos juntos ainda mais preciosos.": "Tristeza", "A tristeza é um mar profundo que às vezes nos engole por inteiro.": "Tristeza", "Às vezes, é preciso chorar para que a dor dentro de nós encontre alívio.": "Tristeza", "A raiva é um fogo que consome tudo em seu caminho se não soubermos controlá-la.": "Raiva", "A injustiça é o combustível que alimenta a chama da revolta.": "Raiva", "A corrupção é como um câncer que se espalha silenciosamente, corroendo tudo ao redor.": "Raiva", "A opressão é uma ferida aberta que sangra a esperança dos oprimidos.": "Raiva", "A impunidade é o veneno que enfraquece a fé na justiça.": "Raiva", "A notícia da sua recuperação trouxe um alívio imenso para todos nós.": "Felicidade"}


In [5]:
df = pd.DataFrame(frases_sentimentos.items(), columns=['Frase', 'Sentimento'])
df.head()

,Frase,Sentimento
0,Estou muito feliz com este resultado!,Felicidade
1,Isso me deixa com muita raiva.,Raiva
2,Sinto-me triste com essa situação.,Tristeza
3,Estou tão animado para este evento!,Felicidade
4,Essa história me deixa arrepiado.,Tristeza


In [6]:
df.shape

(451, 2)

In [7]:
df.duplicated().sum()

0

In [8]:
percentual_sentimentos = df['Sentimento'].value_counts(normalize=True) * 100
percentual_sentimentos


Felicidade    39.689579
Tristeza      32.594235
Raiva         27.716186
Name: Sentimento, dtype: float64

## Preparação dos textos


### Stopwords

Pelo pacote nltk

In [9]:
lista_stopwords = nltk.corpus.stopwords.words('portuguese')
lista_stopwords

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

### Removendo as stopwords

In [10]:
df_stop = df.copy()

In [11]:
def remove_stopwords_from_text(text):
    tokens = word_tokenize(text, language='portuguese')
    filtered_tokens = [word for word in tokens if word.lower() not in lista_stopwords]
    return ' '.join(filtered_tokens)

# Remove as stopwords da coluna 'Frase'
df_stop['Frase Limpa'] = df_stop['Frase'].apply(remove_stopwords_from_text)
df_stop


,Frase,Sentimento,Frase Limpa
0,Estou muito feliz com este resultado!,Felicidade,feliz resultado !
1,Isso me deixa com muita raiva.,Raiva,deixa muita raiva .
2,Sinto-me triste com essa situação.,Tristeza,Sinto-me triste situação .
3,Estou tão animado para este evento!,Felicidade,tão animado evento !
4,Essa história me deixa arrepiado.,Tristeza,história deixa arrepiado .
...,...,...,...
446,A dor da perda é um teste de nossa capacidade ...,Tristeza,dor perda teste capacidade superação crescimen...
447,"A tristeza pode ser uma professora severa, mas...",Tristeza,"tristeza pode professora severa , lições valio..."
448,A corrupção é um veneno que corrói os alicerce...,Raiva,corrupção veneno corrói alicerces qualquer soc...
449,A injustiça é uma chaga que sangra a alma de u...,Raiva,injustiça chaga sangra alma nação .


In [34]:
df_stop.to_csv('stop_words.csv', index=False)

### Representação de texto (Bag of word)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
countvec = CountVectorizer(ngram_range = (1,2))
countvec_data = countvec.fit_transform(df_stop['Frase Limpa'])

In [14]:
# Vocabulário criado a partir dos textos
print(countvec.vocabulary_)

{'feliz': 861, 'resultado': 1727, 'feliz resultado': 865, 'deixa': 548, 'muita': 1257, 'raiva': 1672, 'deixa muita': 555, 'muita raiva': 1258, 'sinto': 1853, 'me': 1198, 'triste': 2040, 'situação': 1866, 'sinto me': 1860, 'me triste': 1199, 'triste situação': 2044, 'tão': 2066, 'animado': 142, 'evento': 799, 'tão animado': 2067, 'animado evento': 143, 'história': 986, 'arrepiado': 195, 'história deixa': 987, 'deixa arrepiado': 550, 'fiquei': 903, 'furioso': 941, 'ouvi': 1398, 'fiquei tão': 904, 'tão furioso': 2072, 'furioso ouvi': 944, 'deprimido': 574, 'notícias': 1317, 'hoje': 990, 'deprimido notícias': 575, 'notícias hoje': 1318, 'radiante': 1661, 'notícia': 1303, 'radiante notícia': 1664, 'injustiça': 1062, 'injustiça deixa': 1067, 'deixa furioso': 552, 'coração': 478, 'doendo': 665, 'tristeza': 2046, 'coração doendo': 481, 'doendo tristeza': 666, 'chocado': 347, 'revelação': 1738, 'chocado revelação': 348, 'transbordando': 2025, 'felicidade': 849, 'transbordando felicidade': 2027,

In [15]:
countvec_data.shape

(451, 2205)

In [20]:
# Conversão da matriz esparsa para DataFrame
df_bow = pd.DataFrame(countvec_data.toarray(), 
                      columns=countvec.get_feature_names_out())
df_bow.index = df_stop.index
df_bow = df_bow.join(df_stop[['Sentimento']], how='left')
df_bow.head()


,aberta,aberta alma,aberta clama,aberta nunca,aberta sangra,aberta sociedade,abraçar,abraçar amamos,abraçar pais,abraço,...,ética dessa,ética moral,última,última fibra,último,último fio,única,única prova,único,Sentimento
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Felicidade
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Raiva
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Tristeza
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Felicidade
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Tristeza


In [21]:
sentimento_map = {'Felicidade': 0, 'Tristeza':1, 'Raiva':2}
df_bow['Sentimento'] = df_bow['Sentimento'].map(sentimento_map)
df_bow.head()

,aberta,aberta alma,aberta clama,aberta nunca,aberta sangra,aberta sociedade,abraçar,abraçar amamos,abraçar pais,abraço,...,ética dessa,ética moral,última,última fibra,último,último fio,única,única prova,único,Sentimento
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Criação do modelo

###  Definição do Target e Feature 

In [22]:
# Target 
y = df_bow['Sentimento']

# Features
x = df_bow.drop('Sentimento', axis=1)

### Divisão de treino e teste

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [24]:
(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

((315, 2205), (136, 2205), (315,), (136,))

##  Criação e treinamento do modelo

In [25]:
modelo = GradientBoostingClassifier(n_estimators=100,
                                        subsample=0.5,
                                        random_state=42)
modelo.fit(x_train, y_train)

GradientBoostingClassifier(random_state=42, subsample=0.5)

In [26]:
y_pred = modelo.predict(x_test)
y_pred

array([1, 2, 1, 0, 2, 0, 1, 1, 0, 1, 0, 1, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0,
       1, 0, 2, 1, 0, 0, 1, 1, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 2, 1, 0, 1, 2, 1, 2, 1, 0, 0, 2, 0, 1, 0, 2, 0, 1, 0,
       1, 1, 0, 0, 0, 2, 0, 1, 1, 2, 0, 1, 0, 0, 2, 1, 2, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 2, 2, 1, 1, 1, 0, 2, 0, 2, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 2, 0, 1, 0, 1, 2, 2, 1, 0, 0, 2, 1, 2, 1, 2, 1, 0, 1, 2, 0,
       1, 1, 0, 0], dtype=int64)

In [27]:
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia:", accuracy)

Acurácia: 0.9485294117647058


## Exportando o modelo

In [28]:
# Exportando o modelo para um arquivo
joblib.dump(modelo, 'modelo.pkl')

['modelo.pkl']

### Testando o modelo com frases inputadas 

In [31]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import CountVectorizer



# Carregando os stop words
df_stop = pd.read_csv('stop_words.csv')

# Carregando o modelo a partir do arquivo
modelo = joblib.load('modelo.pkl')


frase = input("Digite uma frase: ")

# Ajustar o CountVectorizer
countvec = CountVectorizer(ngram_range=(1, 2))
countvec.fit(df_stop['Frase Limpa'])

# Vetorização da frase de teste usando o mesmo CountVectorizer ajustado
frase_vetorizada = countvec.transform([frase])

# Fazer a previsão
classe_prevista = modelo.predict(frase_vetorizada)
print("A frase foi classificada como:", classe_prevista)



Digite uma frase: que dia lindo
A frase foi classificada como: [0]


C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [32]:
df_stop.head()

,Frase Limpa
0,feliz resultado !
1,deixa muita raiva .
2,Sinto-me triste situação .
3,tão animado evento !
4,história deixa arrepiado .
